In [2]:
import numpy as np
import os, yaml, sys
ENV = os.getenv("MY_ENV", "dev")
with open("../../config.yaml", "r") as f:
    config = yaml.safe_load(f)
paths = config[ENV]["paths"]
sys.path.append(paths["src_path"])
from alignment.CKA import cka, cka_minibatch, cka_batch_collection
from sklearn.metrics.pairwise import pairwise_kernels
from alignment.utils import get_usual_transform
from sampling.sampling_methods import random_sets
from parallel.parallel_funcs import print_wise, batch_cka_core
import torch
from torch.utils.data import DataLoader, Subset
import torch.nn.functional as F
from torchvision import datasets, models
from torch.utils.data import Sampler
from torchvision.models.feature_extraction import (
    create_feature_extractor,
    get_graph_node_names,
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
n_batches = 20
batch_size = 10

In [4]:
imagenet_val_path = f"{paths['data_path']}/imagenet/val"
transform = get_usual_transform()
    loader = DataLoader(
        datasets.ImageFolder(imagenet_val_path, transform=transform),
        batch_size=batch_size,
        num_workers=1,
        shuffle=True,
        pin_memory=True,
        timeout=500,
    )

In [5]:
model_names = ["alexnet", "vit_b_16"]
layer_names = ["classifier.5", "encoder.ln"]

In [6]:

model_cls1 = getattr(models, model_names[0])
model1 = model_cls1(pretrained=True).to(device).eval()
model_cls2 = getattr(models, model_names[1])
model2 = model_cls2(pretrained=True).to(device).eval()

/Users/tizianocausin/Desktop/virtual_envs/ponce_env/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/tizianocausin/Desktop/virtual_envs/ponce_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/Users/tizianocausin/Desktop/virtual_envs/ponce_env/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `wei

In [7]:
a = batch_cka_core(1, layer_names, model_names, model1, model2, loader, n_batches, "gram", device, paths)

/Users/tizianocausin/Desktop/virtual_envs/ponce_env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


15:37:28 - rank 1 running batch 1
15:37:32 - rank 1 running batch 2
15:37:36 - rank 1 running batch 3
15:37:40 - rank 1 running batch 4
15:37:43 - rank 1 running batch 5
15:37:46 - rank 1 running batch 6
15:37:49 - rank 1 running batch 7
15:37:53 - rank 1 running batch 8
15:37:57 - rank 1 running batch 9
15:38:00 - rank 1 running batch 10
15:38:04 - rank 1 running batch 11
15:38:07 - rank 1 running batch 12
15:38:11 - rank 1 running batch 13
15:38:14 - rank 1 running batch 14
15:38:18 - rank 1 running batch 15
15:38:22 - rank 1 running batch 16
15:38:25 - rank 1 running batch 17
15:38:29 - rank 1 running batch 18
15:38:33 - rank 1 running batch 19
15:38:37 - rank 1 running batch 20
15:38:46 - rank 1 batch cka : 0.39735320104093363


In [3]:
cka_scores = []
prev_idx = 0
xy, xx, yy = 0, 0, 0
n_pts = X.shape[0]
batch_size = n_pts
batch_num = n_pts // batch_size
idx = np.random.choice(np.arange(n_pts), size=n_pts, replace=False)
prev_idx = 0
counter = 0
for i in range(batch_size, n_pts+batch_size, batch_size):  # iterate over minibatches
    i = min(i, n_pts)
    #X_batch, Y_batch = X_batch.cuda(), Y_batch.cuda()
    #idx = np.random.choice(np.arange(n_pts), size=batch_size, replace=False)
    #X_batch, Y_batch = X[idx, :], Y[idx, :]
    #print("batch start", prev_idx, "batch end", i-1)
    counter +=1
    #print(counter)
    rand_idx = idx[prev_idx:i]
    X_batch, Y_batch = X[rand_idx, :], Y[rand_idx, :]
    prev_idx = i
    xy_n, xx_n, yy_n = cka_minibatch(X_batch, Y_batch, kernel="linear")
    xy += xy_n
    xx += xx_n
    yy += yy_n
#cka_value = np.mean(np.stack(cka_scores))
print("Estimated minibatch CKA:", cka_batch_collection(xy, xx, yy))

Estimated minibatch CKA: 0.8620530945189993
